In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

plt.rcParams["svg.fonttype"] = "none"
plt.rcParams["pdf.fonttype"] = 42

import seaborn as sns
sns.set_theme(style="dark")

from scripts.paper_figures.data_processing.data_processing import main as init_data
from scripts.paper_figures.data_processing.process_pumping import ProboscisProcessing


In [2]:
llh, processed_data , io_process = init_data()

Data Loading and Preprocessing...


AttributeError: 'Args' object has no attribute 'TMP_RESULT_FOLDER'

In [3]:
from filter_timeseries import ProboscisAnalysis
import filter_timeseries
import importlib

importlib.reload(filter_timeseries)
from filter_timeseries import ProboscisAnalysis

prob_dat = ProboscisAnalysis(bouts_dict)
proc_bouts = prob_dat.preprocess_all_data()
import filter_timeseries
import importlib

importlib.reload(filter_timeseries)
from filter_timeseries import ProboscisAnalysis

shortest_length = prob_dat.find_shortest_series('distance.origin-prob')

# Now, you can segment each time series in each dataframe in your data dictionary
for key in bouts_dict.keys():
    bouts_dict[key] = prob_dat.segment_timeseries_column(bouts_dict[key], 'distance.origin-prob', shortest_length)
unpacked_df = prob_dat.unpack_dataframes('distance.origin-prob_segmented')
unpacked_df = ProboscisAnalysis.calculate_spectral_density_for_dataframe(unpacked_df)
import plotly.subplots as sp
import plotly.graph_objects as go

# Assuming your DataFrame is named `df` and contains the 'time_series' and 'PSDs' columns

# Select the row you want to plot (e.g., row with index 0)
row_index = 15
selected_row = unpacked_df.iloc[row_index]

# Extract the time series and PSDs from the selected row
time_series = selected_row['time_series']
psds = selected_row['PSDs']

# Create subplots with shared x-axis
fig = sp.make_subplots(rows=1, cols=2, shared_xaxes=True)

# Add the time series subplot as a line plot
fig.add_trace(go.Scatter(x=list(range(len(time_series))), y=time_series, mode='lines', name='Time Series'), row=1,
              col=1)

# Add the PSDs subplot as a line plot
fig.add_trace(go.Scatter(x=psds[0], y=psds[1], mode='lines', name='PSDs'), row=1, col=2)

# Update subplot titles and layout
fig.update_layout(
    title='Time Series and Power Spectral Density',
    xaxis=dict(title='Time/Frequency'),
    yaxis1=dict(title='Value', domain=[0.2, 1]),
    yaxis2=dict(title='Power', domain=[0, 0.8]),
    width=800,
    height=400,
    showlegend=True,
    legend=dict(x=0, y=1),
)

# Show the plot
fig.show()

import numpy as np
import matplotlib.pyplot as plt

# Define the number of bins for the histogram
n_bins = 50

# Initialize a list to store the histogram for each instance
histograms = []

# For each instance in the dataset
for psd in unpacked_df['PSDs']:
    # Get the frequencies and powers
    frequencies = psd[0]
    powers = psd[1]

    # Filter out frequencies higher than 5 Hz
    powers = [power for freq, power in zip(frequencies, powers) if freq <= 5]

    # Compute the histogram of the powers
    hist, bin_edges = np.histogram(powers, bins=n_bins)

    # Normalize the histogram
    hist = hist / np.sum(hist)

    # Add the histogram to the list
    histograms.append(hist)

# Convert the list of histograms into a 2D array
histograms = np.array(histograms)

# Now, 'histograms' is a 2D array where each row is an instance and each column is a histogram bin.
# You can use this array as input to your machine learning algorithm.

from sklearn.decomposition import PCA

# Create a PCA object
pca = PCA(n_components=3)  # Change this to the number of components you want

# Fit the PCA model to your data and apply the dimensionality reduction
pca_result = pca.fit_transform(histograms)

# Print the explained variance ratio
print('Explained variance ratio:', pca.explained_variance_ratio_)

import matplotlib.pyplot as plt

# Get the weights of the original features for the first principal component
component = pca.components_[0]

# Create a list of frequency bands corresponding to your histogram bins
frequency_bands = [f"{0.1 * i}-{0.1 * (i + 1)}Hz" for i in range(50)]

# Create a bar plot
plt.figure(figsize=(15, 5))  # Increase the figure size for better visibility
plt.bar(frequency_bands, component)
plt.xlabel('Frequency Band')
plt.ylabel('Importance')
plt.title('Importance of Frequency Bands for First Principal Component')
plt.xticks(rotation=90)  # Rotate the x-axis labels for better visibility
plt.show()

print('Components:', pca.components_)
expt_name = 'Fly06292022_5d_A'
#plt.plot(bouts_dict[expt_name]['distance.origin-prob_segmented'][1][200])
len(bouts_dict[expt_name]['distance.origin-prob_segmented'][1])
pds2 = spect_density_dict[expt_name]['PSD_50']
psd = (spect_density_dict[expt_name]['PSD_50'][50])

import plotly.express as px

# Plot the spectral density using Plotly
fig = px.line(
    x=psd[0],
    y=psd[1],
    title="PSD for",
    width=700,
    height=400
)
fig.show()
len(bouts_dict[expt_name]['distance.origin-prob_segmented'][2])
expt_name = 'Fly06292022_5d_A'
plt.plot(bouts_dict[expt_name]['distance.origin-prob_segmented'][1][1])
expt_name = 'Fly06292022_5d_A'

dt = np.diff(bouts_dict[expt_name]['distance.origin-prob_segmented'][1][1])
#plt.plot(bouts_dict[expt_name]['distance.origin-prob'][9])
plt.plot(dt)
sig = bouts_dict[expt_name]['distance.origin-prob']
print(type(sig.iloc[0]))
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import stft

# let's assume your dictionary is named `experiments`
# get the first key and dataframe from the dictionary
first_key = list(bouts_dict.keys())[0]
first_key = 'Fly06292022_5d_A'
first_df = bouts_dict[first_key]

# choose the time series feature you want to transform
feature = 'distance.origin-prob'

# perform STFT on each row
for i, row in first_df.iterrows():
    f, t, Zxx = stft(row[feature], fs=30, nperseg=64, noverlap=32)  # fs=30Hz is the sample rate

    # Plot the spectrogram
    plt.pcolormesh(t, f, np.abs(Zxx), shading='gouraud')
    plt.title(f'Spectrogram for row {i + 1} in experiment {first_key}')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.ylim([0.2, 1.1])
    plt.pcolormesh(t, f, 10 * np.log10(np.abs(Zxx)), shading='gouraud')
    plt.colorbar(label='dB')

    plt.show()

    # To avoid plotting all rows, break the loop after first one (remove this to plot all rows)



[]